# Przygotowanie

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import StratifiedKFold

from nltk.tokenize import TweetTokenizer

In [2]:
from const import PREPROCESSED_TEST_PATH, PREPROCESSED_TRAIN_PATH

# Dane

In [3]:
df_train = pd.read_pickle(PREPROCESSED_TRAIN_PATH)
df_test = pd.read_pickle(PREPROCESSED_TEST_PATH)

In [4]:
Y_train = df_train.target.to_numpy()
Y_test = df_test.target.to_numpy()

In [5]:
X_train = df_train.text.to_numpy()
X_test = df_test.text.to_numpy()

Podobnie jak w przypadku sieci neuronowych musimy zwektoryzować nasz teks. W tym celu możemy wykorzystać __word2vec__. Jednak w tym przykładzie postanowiłem skorzystać z __TfidfVectorizer__, którego niestety z przyczyn wydajnościowych musiałem zamienić na __CountVectorizer__.

In [6]:
# Ten kod jest nieużywane w dalszej części notebooka.
# Pokazuje on jedynie inną metodę konwersji danych do wektora.

tfv = TfidfVectorizer(sublinear_tf=True)
tfv.fit(df_train.text)

X_train_t = tfv.transform(df_train.text)
X_test_t = tfv.transform(df_test.text)

Aby użyć __CountVectorizer__ poprawnie, należy dodać funkcję, która zamienia lity tekst na tokeny. Postanowiłem skorzystać z klasy znajdującej się w __nltk__ - __TweetTokenizer__. Wybrałem ją w oparciu o porównanie znajdujące się https://towardsdatascience.com/an-introduction-to-tweettokenizer-for-processing-tweets-9879389f8fe7

In [6]:
def tokenize(text): 
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 1))

Możemy teraz stowrzyć teraz połączyć model z vectorizerem za pomocą pipelinu.

In [7]:
pipeline_svm = make_pipeline(vectorizer, 
                            SVC(probability = True))

In [8]:
grid_svm = GridSearchCV(pipeline_svm,
                        param_grid = {'svc__C': [0.01, 0.1, 1, 10, 100, 1000]}, 
                        cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state =1 ),
                        verbose = 10,   
                        n_jobs = -1) 

# Uczenie

In [9]:
grid_svm.fit(X_train, Y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   36.1s remaining:   54.2s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   36.1s remaining:   24.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.1min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('countvectorizer',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        

In [10]:
grid_svm.score(X_test, Y_test)

0.72670625

In [11]:
pickle.dump(grid_svm.best_estimator_, open("SVM.model", 'wb'))

In [12]:
grid_svm.best_params_

{'svc__C': 1}